In [1]:
import numpy as np
from wsvm.application import Wsvm
from svm.application import Svm
from sklearn.svm import SVC
#from sklearn.metrics import f1_score
from sklearn.metrics  import classification_report,precision_recall_fscore_support as score
from sklearn.metrics import accuracy_score, confusion_matrix,roc_auc_score,f1_score
from sklearn.neighbors import NearestNeighbors
from sklearn.utils import _safe_indexing
from sklearn import metrics
import math
from datetime import datetime
from fuzzy.weight import fuzzy
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import StandardScaler
import csv

In [2]:
def svm_lib(X_train, y_train,X_test):
    svc=SVC(probability=True, kernel='linear')
    model = svc.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    return y_pred

In [3]:
def wsvm(C,X_train, y_train,X_test,distribution_weight=None):
    model = Wsvm(C,distribution_weight)
    model.fit(X_train, y_train)
    test_pred = model.predict(X_test)
    return test_pred

In [4]:
def svm(C,X_train, y_train,X_test):
    model = Svm(C)
    model.fit(X_train, y_train)
    test_pred = model.predict(X_test)
    return test_pred

In [5]:
def is_tomek_new(X,y, class_type):
    # print(y)
    # print(type(y))
    nn = NearestNeighbors(n_neighbors=2)
    nn.fit(X)
    nn_index = nn.kneighbors(X, return_distance=False)[:, 1]
    links = np.zeros(len(y), dtype=bool)
    # find which class to not consider
    class_excluded = [c for c in np.unique(y) if c not in class_type]
    X_dangxet = []
    X_tl = []
    # there is a Tomek link between two samples if they are both nearest
    # neighbors of each others.
    for index_sample, target_sample in enumerate(y):
        if target_sample in class_excluded:
            continue
        if y[nn_index[index_sample]] != target_sample:
            if nn_index[nn_index[index_sample]] == index_sample:
                X_tl.append(index_sample)
                X_dangxet.append(nn_index[index_sample])
                links[index_sample] = True

    stt = np.zeros(len(X_dangxet), dtype=int)
    arr_tlp = np.stack((X_dangxet, X_tl, stt), axis=1)

    return arr_tlp

In [6]:
def Gmean(y_test,y_pred):
    cm_WSVM = metrics.confusion_matrix(y_test, y_pred)
    sensitivity = cm_WSVM[1,1]/(cm_WSVM[1,0]+cm_WSVM[1,1])
    specificity = cm_WSVM[0,0]/(cm_WSVM[0,0]+cm_WSVM[0,1])
    gmean = math.sqrt(sensitivity*specificity)
    return specificity,sensitivity,gmean

In [7]:
def compute_weight(X, y,name_method ="actual_hyper_lin", name_function = "exp", beta = None,C = None, gamma = None, u = None, sigma = None):
    method = fuzzy.method()
    function = fuzzy.function()
    pos_index = np.where(y == 1)[0]
    neg_index = np.where(y == -1)[0]
    try:
        if name_method == "own_class_center": 
            d = method.own_class_center(X, y)
        elif name_method == "estimated_hyper_lin": # actual_hyper_lin, own_class_center
            d = method.estimated_hyper_lin(X, y)
        elif name_method == "own_class_center_opposite":
            d = method.own_class_center_opposite(X, y)
        elif name_method == 'actual_hyper_lin':
            d = method.actual_hyper_lin(X, y,C = C, gamma = gamma)
        elif name_method == 'own_class_center_divided':
            d = method.own_class_center_divided(X, y)
        elif name_method == "distance_center_own_opposite_tam":
            d_own, d_opp, d_tam = method.distance_center_own_opposite_tam(X,y)
        else:
            print('dont exist method')
        
        if name_function == "lin":
            W = function.lin(d)
        elif name_function == "exp":
            W = function.exp(d, beta)
        elif name_function == "lin_center_own":
            W = function.lin_center_own(d, pos_index,neg_index)
        elif name_function == 'gau':
            W = function.gau(d, u, sigma)
        elif name_function == "func_own_opp_new":
            W = function.func_own_opp_new(d_own,d_opp,pos_index,neg_index,d_tam)
        elif name_function == "func_own_opp_new_v1":
            W = function.func_own_opp_new_v1(d_own,d_opp,pos_index,neg_index,d_tam)
        elif name_function == "func_own_opp_new_v2":
            W = function.func_own_opp_new_v2(d_own,d_opp,pos_index,neg_index,d_tam)
    except Exception as e:
        print('dont exist function')
        print(e)
    # pos_index = np.where(y == 1)[0]
    # neg_index = np.where(y == -1)[0]
    r_pos = 1
    r_neg = len(pos_index)/len(neg_index)
    m = []
    W = np.array(W)
    m = W[pos_index]*r_pos
    m = np.append(m, W[neg_index]*r_neg)
    return m

In [8]:
def fuzzy_weight(beta_center, beta_estimate, beta_actual,X_train, y_train,namemethod,namefunction):
    if namemethod =="own_class_center_opposite" and namefunction == "exp":
        distribution_weight = compute_weight(X_train, y_train,name_method = namemethod,name_function = namefunction,beta = beta_center)
    elif namemethod =="own_class_center" and namefunction == "exp":
        distribution_weight = compute_weight(X_train, y_train,name_method = namemethod,name_function = namefunction,beta = beta_estimate)
    elif namemethod =="own_class_center_divided" and namefunction == "exp":
        distribution_weight = compute_weight(X_train, y_train,name_method = namemethod,name_function = namefunction,beta = beta_estimate)
    elif namemethod =="estimated_hyper_lin" and namefunction == "exp":
        distribution_weight = compute_weight(X_train, y_train,name_method = namemethod,name_function = namefunction,beta = beta_estimate)
    elif namemethod =="actual_hyper_lin" and namefunction == "exp":
        distribution_weight = compute_weight(X_train, y_train,name_method = namemethod,name_function = namefunction,beta = beta_actual)
    else:   
        distribution_weight = compute_weight(X_train, y_train,name_method = namemethod,name_function = namefunction)
    return distribution_weight

In [9]:
# Ban đầu
def data_tomelinks_new1(C,weight,X_test,y_test,X_train,y_train,n_neighbors,arr_tlp,clf=None,namemethod=None,namefunction=None):
    ro1 = 0.1
    ro3 = 0.3
    ro4 = 0.5
    ro2 = 0.5
    # links,ind_posX,ind_negX = is_tomek(X_train,y_train,class_type=[-1.0])
    
    #print(len(ind_posX))
    new_W = weight
    pos_index = np.where(y_train == 1)[0]
    neg_index = np.where(y_train == -1)[0]
    clf = Wsvm(C,new_W)
    clf.fit(X_train, y_train)    
    nn2 = NearestNeighbors(n_neighbors=n_neighbors)
    nn2.fit(X_train)

    # Mẫu âm bị phân loại sai -> Giảm trọng số của mẫu âm đó
    neg_pred = clf.predict(X_train[neg_index])
    idx_neg_wrong = np.where(neg_pred != -1.0)
    new_W[idx_neg_wrong] =  new_W[idx_neg_wrong]*0.5 # giam manh (Co-Author*0.9)

    # Tăng, giảm trọng số của các mẫu trong TLPs
    # Trường hợp 1, 2, 3, 4
    ind_nn_pos = [] # chứa chỉ số của các mẫu dương bị phân loại sai trong ind_posX
    y_nn_pos = [] #chứa nhãn của k mẫu dữ liệu gần nhất với mẫu dương được xét
    ind_nn_neg = [] # chứa chỉ số của các mẫu âm bị phân loại sai trong ind_negX
    y_nn_neg = [] # chứa nhãn của k mẫu dữ liệu gần nhất với mẫu âm được xét

    for ind,i in enumerate(arr_tlp):
        y_pred_pos = clf.predict([X_train[arr_tlp[ind][0]]]) #positive
        y_pred_neg = clf.predict([X_train[arr_tlp[ind][1]]]) #negative
        if (y_pred_pos == 1) and (y_pred_neg == 1): #dương dự đoán đúng, âm dự đoán sai
            new_W[arr_tlp[ind][0]] = new_W[arr_tlp[ind][0]]*(1 + ro1) # tăng trọng số mẫu dương
            new_W[arr_tlp[ind][1]] = new_W[arr_tlp[ind][1]]*(1 - ro1) # giảm trọng số mẫu âm
            arr_tlp[ind][2] = 1

            ind_nn_neg.append(ind)                          
            knn_X = (nn2.kneighbors([X_train[arr_tlp[ind][1]]])[1]).tolist()  #Xem lại
            for j in knn_X[0]:
                y_nn_neg.append(y_train[j])    # gom nhãn láng giềng của X_train[i] bị dự đoán sai vào y_nn_neg

        if (y_pred_pos == -1) and (y_pred_neg == -1): #âm dự đoán đúng, dương dự đoán sai
            new_W[arr_tlp[ind][0]] = new_W[arr_tlp[ind][0]]*(1 + ro3) 
            new_W[arr_tlp[ind][1]] = new_W[arr_tlp[ind][1]]*(1 - ro3)
            # new_W[arr_tlp[ind][1]] = new_W[arr_tlp[ind][1]]
            arr_tlp[ind][2] = 3

            ind_nn_pos.append(ind)                          
            knn_X = (nn2.kneighbors([X_train[arr_tlp[ind][0]]])[1]).tolist() #Xem lại 
            for j in knn_X[0]:
                y_nn_pos.append(y_train[j])    # gom nhãn láng giềng của X_train[i] bị dự đoán sai vào y_nn

    ind_nn_neg = np.array(ind_nn_neg)
    y_nn_neg = np.array(y_nn_neg)
    if len(y_nn_neg)>0:
        y_nn_neg = np.array_split(y_nn_neg, len(y_nn_neg)/n_neighbors) 
        for ind,i in enumerate(range(0,len(y_nn_neg))):   #
            if -1 not in y_nn_neg[i][1:]:      # Nếu không tồn tại nhãn -1 xung quanh X_train[i] bị dự đoán sai => nhiễu âm -> giảm mạnh trọng số
                for a in arr_tlp[[ind_nn_neg[ind]]]:
                    new_W[a[1]] = new_W[a[1]]*ro2
                arr_tlp[ind][2] = 2

    ind_nn_pos = np.array(ind_nn_pos)
    y_nn_pos = np.array(y_nn_pos)
    if len(y_nn_pos)>0:
        y_nn_pos = np.array_split(y_nn_pos, len(y_nn_pos)/n_neighbors) 
        for ind,i in enumerate(range(0,len(y_nn_pos))):   #
            if 1 not in y_nn_pos[i][1:]:      # Nếu không tồn tại nhãn 1 xung quanh X_train[i] bị dự đoán sai => nhiễu dương -> giảm mạnh trọng số
                for a in arr_tlp[[ind_nn_pos[ind]]]:
                    new_W[a[0]] = new_W[a[0]]*ro4
                arr_tlp[ind][2] = 4

    return new_W

In [10]:
def data_tomelinks_modify(ro1,ro2,C,weight,X_test,y_test,X_train,y_train,n_neighbors,arr_tlp,clf=None,namemethod=None,namefunction=None):

    # links,ind_posX,ind_negX = is_tomek(X_train,y_train,class_type=[-1.0])
    
    #print(len(ind_posX))
    new_W = weight
    pos_index = np.where(y_train == 1)[0]
    neg_index = np.where(y_train == -1)[0]
    clf = Wsvm(C,new_W)
    clf.fit(X_train, y_train)    
    nn2 = NearestNeighbors(n_neighbors=n_neighbors)
    nn2.fit(X_train)

    # Mẫu âm bị phân loại sai -> Giảm trọng số của mẫu âm đó
    neg_pred = clf.predict(X_train[neg_index])
    idx_neg_wrong = np.where(neg_pred != -1.0)
    new_W[idx_neg_wrong] =  new_W[idx_neg_wrong]*0.5 # giam manh (Co-Author*0.9)

    # Tăng, giảm trọng số của các mẫu trong TLPs
    # Trường hợp 1, 2, 3, 4
    ind_nn_pos = [] # chứa chỉ số của các mẫu dương bị phân loại sai trong ind_posX
    y_nn_pos = [] #chứa nhãn của k mẫu dữ liệu gần nhất với mẫu dương được xét
    ind_nn_neg = [] # chứa chỉ số của các mẫu âm bị phân loại sai trong ind_negX
    y_nn_neg = [] # chứa nhãn của k mẫu dữ liệu gần nhất với mẫu âm được xét

    for ind,i in enumerate(arr_tlp):
        y_pred_pos = clf.predict([X_train[arr_tlp[ind][0]]]) #positive
        y_pred_neg = clf.predict([X_train[arr_tlp[ind][1]]]) #negative
        if (y_pred_pos == 1) and (y_pred_neg == 1): #dương dự đoán đúng, âm dự đoán sai
            new_W[arr_tlp[ind][0]] = new_W[arr_tlp[ind][0]]*(1 + ro1) # tăng trọng số mẫu dương
            new_W[arr_tlp[ind][1]] = new_W[arr_tlp[ind][1]]*(1 - ro1) # giảm trọng số mẫu âm
            arr_tlp[ind][2] = 1

            ind_nn_neg.append(ind)                          
            knn_X = (nn2.kneighbors([X_train[arr_tlp[ind][1]]])[1]).tolist()  #Xem lại
            for j in knn_X[0]:
                y_nn_neg.append(y_train[j])    # gom nhãn láng giềng của X_train[i] bị dự đoán sai vào y_nn_neg

        if (y_pred_pos == -1) and (y_pred_neg == -1): #âm dự đoán đúng, dương dự đoán sai
            new_W[arr_tlp[ind][0]] = new_W[arr_tlp[ind][0]]*(1 + ro1) 
            new_W[arr_tlp[ind][1]] = new_W[arr_tlp[ind][1]]*(1 - ro1)
            arr_tlp[ind][2] = 3

            ind_nn_pos.append(ind)                          
            knn_X = (nn2.kneighbors([X_train[arr_tlp[ind][0]]])[1]).tolist() #Xem lại 
            for j in knn_X[0]:
                y_nn_pos.append(y_train[j])    # gom nhãn láng giềng của X_train[i] bị dự đoán sai vào y_nn

    ind_nn_neg = np.array(ind_nn_neg)
    y_nn_neg = np.array(y_nn_neg)
    if len(y_nn_neg)>0:
        y_nn_neg = np.array_split(y_nn_neg, len(y_nn_neg)/n_neighbors) 
        for ind,i in enumerate(range(0,len(y_nn_neg))):   #
            if -1 not in y_nn_neg[i][1:]:      # Nếu không tồn tại nhãn -1 xung quanh X_train[i] bị dự đoán sai => nhiễu âm -> giảm mạnh trọng số
                for a in arr_tlp[[ind_nn_neg[ind]]]:
                    new_W[a[1]] = new_W[a[1]]*ro2
                arr_tlp[ind][2] = 2

    ind_nn_pos = np.array(ind_nn_pos)
    y_nn_pos = np.array(y_nn_pos)
    if len(y_nn_pos)>0:
        y_nn_pos = np.array_split(y_nn_pos, len(y_nn_pos)/n_neighbors) 
        for ind,i in enumerate(range(0,len(y_nn_pos))):   #
            if 1 not in y_nn_pos[i][1:]:      # Nếu không tồn tại nhãn 1 xung quanh X_train[i] bị dự đoán sai => nhiễu dương -> giảm mạnh trọng số
                for a in arr_tlp[[ind_nn_pos[ind]]]:
                    new_W[a[0]] = new_W[a[0]]*ro2
                arr_tlp[ind][2] = 4

    return new_W

In [11]:
name_method =["own_class_center","estimated_hyper_lin","actual_hyper_lin","distance_center_own_opposite_tam"]
name_function = ["lin_center_own","exp","func_own_opp_new", "func_own_opp_new_v1", "func_own_opp_new_v2"]
for namemethod in name_method:
    for namefunction in name_function:
        if namemethod =="distance_center_own_opposite_tam" and namefunction =="lin_center_own":
            continue
        elif namemethod =="distance_center_own_opposite_tam" and namefunction =="exp":
            continue
        elif namemethod == "own_class_center" and namefunction == "func_own_opp_new":
            continue
        elif namemethod == "estimated_hyper_lin" and namefunction == "func_own_opp_new":
            continue
        elif namemethod == "actual_hyper_lin" and namefunction == "func_own_opp_new":
            continue
        elif namemethod == "own_class_center" and namefunction == "func_own_opp_new_v1":
            continue
        elif namemethod == "estimated_hyper_lin" and namefunction == "func_own_opp_new_v1":
            continue
        elif namemethod == "actual_hyper_lin" and namefunction == "func_own_opp_new_v1":
            continue
        elif namemethod == "own_class_center" and namefunction == "func_own_opp_new_v2":
            continue
        elif namemethod == "estimated_hyper_lin" and namefunction == "func_own_opp_new_v2":
            continue
        elif namemethod == "actual_hyper_lin" and namefunction == "func_own_opp_new_v2":
            continue
        # elif namemethod == "distance_center_own_opposite_tam" and namefunction == "lin":
        #     continue
        # elif namemethod == "own_class_center" and namefunction == "lin":
        #     continue
        # elif namemethod == "estimated_hyper_lin" and namefunction == "lin":
        #     continue
        # elif namemethod == "actual_hyper_lin" and namefunction == "lin_center_own":
        #    continue
        # elif namemethod == "own_class_center" and namefunction == "lin_center_own":
        #     continue 
        # elif namemethod == "own_class_center" and namefunction == "exp":
        #     continue 
        # elif namemethod == "estimated_hyper_lin" and namefunction == "lin_center_own":
        #     continue 
        # elif namemethod == "estimated_hyper_lin" and namefunction == "exp":
        #     continue 
        # elif namemethod == "actual_hyper_lin" and namefunction == "lin_center_own":
        #     continue
        # elif namemethod == "actual_hyper_lin" and namefunction == "exp":
        #     continue
        else:
            print(namemethod, namefunction)

own_class_center lin_center_own
own_class_center exp
estimated_hyper_lin lin_center_own
estimated_hyper_lin exp
actual_hyper_lin lin_center_own
actual_hyper_lin exp
distance_center_own_opposite_tam func_own_opp_new
distance_center_own_opposite_tam func_own_opp_new_v1
distance_center_own_opposite_tam func_own_opp_new_v2


In [12]:
name_method =["own_class_center","estimated_hyper_lin","actual_hyper_lin","distance_center_own_opposite_tam"]
name_function = ["lin_center_own","exp","func_own_opp_new_v2"]
for namemethod in name_method:
    for namefunction in name_function:
        if namemethod =="distance_center_own_opposite_tam" and namefunction =="lin_center_own":
            continue
        elif namemethod =="distance_center_own_opposite_tam" and namefunction =="exp":
            continue
        elif namemethod == "own_class_center" and namefunction == "func_own_opp_new_v2":
            continue
        elif namemethod == "estimated_hyper_lin" and namefunction == "func_own_opp_new_v2":
            continue
        elif namemethod == "actual_hyper_lin" and namefunction == "func_own_opp_new_v2":
            continue
        elif namemethod == "distance_center_own_opposite_tam" and namefunction == "func_own_opp_new_v2":
            continue
        else:
            print(namemethod, namefunction)

own_class_center lin_center_own
own_class_center exp
estimated_hyper_lin lin_center_own
estimated_hyper_lin exp
actual_hyper_lin lin_center_own
actual_hyper_lin exp


In [13]:
################################# K-FOLD SCRIPT FOR CO-AUTHOR DATASETS-MAIN ##############################
C = 100
T = 15
N = 3
n_neighbor = 5

from ProcessingData import ACOMP_CoAuthor
dataset = ACOMP_CoAuthor
beta_center, beta_estimate, beta_actual = 0.6, 1, 0.3
# beta_center, beta_estimate, beta_actual = 0.8, 0.5, 0.5

name_method =["own_class_center","estimated_hyper_lin","actual_hyper_lin","distance_center_own_opposite_tam"]
name_function = ["lin_center_own","exp","func_own_opp_new_v2"]

time = datetime.now().strftime("%d%m%Y_%H%M%S")
# filename = (str(dataset).split("\\")[-1]).split(".")[0]
filename = 'ACOMP_21_G3_0.08_36_600_6'

#W.svm
for n in range(0,N):
    header = ['Times','Fold','T','Name Method', 'Name Function', 'SP', 'SE', 'Gmean', 'F1 Score','Accuracy','AUC','Ma tran nham lan']
    X, y = dataset.load_data()
    print(X.shape)
    kfold_validation = StratifiedKFold(n_splits=5, shuffle=True) 
    with open(f'./Experiment/{filename}_{time}.csv', 'a', encoding='UTF8', newline='') as f3:
        writer = csv.writer(f3)
        writer.writerow(header)
        fold = 1
        for train_index, test_index in kfold_validation.split(X,y):
            print("\nN = ",n+1,", Fold = ",fold)
            # X_train, y_train = X.iloc[train_index], y.iloc[train_index]
            # X_test, y_test = X.iloc[test_index], y.iloc[test_index]
            X_train, y_train = X[train_index], y[train_index]
            X_test, y_test = X[test_index], y[test_index]
            # print(X_test.shape)
            
            #Scalling Data
            sc_X = StandardScaler()
            X_train = sc_X.fit_transform(X_train)
            X_test = sc_X.transform(X_test)
            y_train = np.array(y_train)

            # NORMAL

            #Svm library
            print("SVM LIBRARY starting...")
            test_pred = svm_lib(X_train, y_train,X_test)
            test_pred = np.nan_to_num(test_pred)
            sp,se,gmean = Gmean(y_test,test_pred)
            name1 = 'SVM'
            name2 = 'SVM'
            writer.writerow([n+1,fold,"None",name1,name2,sp,se,gmean,f1_score(y_test, test_pred),accuracy_score(y_test,test_pred),roc_auc_score(y_test, test_pred),str(confusion_matrix(y_test, test_pred))])
            
            
            #Wsvm
            print("W.SVM starting...")
            N, d = X_train.shape
            distribution_weight = np.ones(N)
            test_pred = wsvm(C,X_train, y_train, X_test, distribution_weight)
            test_pred = np.nan_to_num(test_pred)
            sp,se,gmean = Gmean(y_test,test_pred)
            name1 = 'WSVM'
            name2 = 'WSVM'
            writer.writerow([n+1,fold,"None",name1,name2,sp,se,gmean,f1_score(y_test, test_pred),accuracy_score(y_test,test_pred),roc_auc_score(y_test, test_pred),str(confusion_matrix(y_test, test_pred))])

            #FuzyyWsvm
            for namemethod in name_method:
                for namefunction in name_function:
                    if namemethod =="distance_center_own_opposite_tam" and namefunction =="lin_center_own":
                        continue
                    elif namemethod =="distance_center_own_opposite_tam" and namefunction =="exp":
                        continue
                    elif namemethod == "own_class_center" and namefunction == "func_own_opp_new_v2":
                        continue
                    elif namemethod == "estimated_hyper_lin" and namefunction == "func_own_opp_new_v2":
                        continue
                    elif namemethod == "actual_hyper_lin" and namefunction == "func_own_opp_new_v2":
                        continue
                    elif namemethod == "distance_center_own_opposite_tam" and namefunction == "func_own_opp_new_v2":
                        continue
                    else:
                        print(f"Fuzzy W.SVM name_method = '{namemethod}',name_function = '{namefunction}' starting...")
                        distribution_weight = fuzzy_weight(beta_center, beta_estimate, beta_actual,X_train, y_train,namemethod,namefunction)
                        __ = fuzzy_weight(beta_center, beta_estimate, beta_actual,X_train, y_train,namemethod,namefunction)
                        test_pred = wsvm(C,X_train, y_train, X_test, distribution_weight)
                        test_pred = np.nan_to_num(test_pred)
                        sp,se,gmean = Gmean(y_test,test_pred)
                        writer.writerow([n+1,fold,"None",namemethod,namefunction,sp,se,gmean,f1_score(y_test, test_pred),accuracy_score(y_test,test_pred),roc_auc_score(y_test, test_pred),str(confusion_matrix(y_test, test_pred))])

            ############ USING AFW-CIL METHOD #########################
            print("===========USING AFW-CIL=====================")
            arr_tlp = is_tomek_new(X_train, y_train, class_type = [-1.0])
            #FuzyyWsvm
            for namemethod in name_method:
                for namefunction in name_function:
                    if namemethod =="distance_center_own_opposite_tam" and namefunction =="lin_center_own":
                        continue
                    elif namemethod =="distance_center_own_opposite_tam" and namefunction =="exp":
                        continue
                    elif namemethod == "own_class_center" and namefunction == "func_own_opp_new_v2":
                        continue
                    elif namemethod == "estimated_hyper_lin" and namefunction == "func_own_opp_new_v2":
                        continue
                    elif namemethod == "actual_hyper_lin" and namefunction == "func_own_opp_new_v2":
                        continue
                    elif namemethod == "distance_center_own_opposite_tam" and namefunction == "func_own_opp_new_v2":
                        continue
                    else:
                        print(f"Fuzzy W.SVM name_method = '{namemethod}',name_function = '{namefunction}' starting...")
                        distribution_weight = fuzzy_weight(beta_center, beta_estimate, beta_actual,X_train, y_train,namemethod,namefunction)
                        __ = fuzzy_weight(beta_center, beta_estimate, beta_actual,X_train, y_train,namemethod,namefunction)
                        
                        for i in range(0,T):
                            print("T = ",i+1)
                            new_W = data_tomelinks_new1(C,distribution_weight,X_test,y_test,X_train,y_train,n_neighbor,arr_tlp,clf=None,namemethod=namemethod,namefunction=namefunction)
                            test_pred = wsvm(C,X_train, y_train, X_test, new_W)
                            test_pred = np.nan_to_num(test_pred)
                            sp,se,gmean = Gmean(y_test,test_pred)
                            writer.writerow([n+1,fold,i+1,namemethod,namefunction,sp,se,gmean,f1_score(y_test, test_pred),accuracy_score(y_test,test_pred),roc_auc_score(y_test, test_pred),str(confusion_matrix(y_test, test_pred))])
            fold = fold + 1

(600, 7)

N =  1 , Fold =  1
SVM LIBRARY starting...
W.SVM starting...


Fuzzy W.SVM name_method = 'own_class_center',name_function = 'lin_center_own' starting...
Fuzzy W.SVM name_method = 'own_class_center',name_function = 'exp' starting...
Fuzzy W.SVM name_method = 'estimated_hyper_lin',name_function = 'lin_center_own' starting...
Fuzzy W.SVM name_method = 'estimated_hyper_lin',name_function = 'exp' starting...
Fuzzy W.SVM name_method = 'actual_hyper_lin',name_function = 'lin_center_own' starting...
Fuzzy W.SVM name_method = 'actual_hyper_lin',name_function = 'exp' starting...
===========USING AFW-CIL=====================
Fuzzy W.SVM name_method = 'own_class_center',name_function = 'lin_center_own' starting...
T =  1
T =  2
T =  3
T =  4
T =  5
T =  6
T =  7
T =  8
T =  9
T =  10
T =  11
T =  12
T =  13
T =  14
T =  15
Fuzzy W.SVM name_method = 'own_class_center',name_function = 'exp' starting...
T =  1
T =  2
T =  3
T =  4
T =  5
T =  6
T =  7
T =  8
T =  9
T =  10
T =  11
T =  12
T =  13
T =  14
T =  15
Fuzzy W.SVM name_method = 'estimated_hyper_lin',na

In [14]:
# ################################# K-FOLD SCRIPT FOR CO-AUTHOR DATASETS-MODIFY ##############################
# C = 1000
# T = 20
# N = 3
# # Ro1 = [0.1,0.3,0.5,0.7,0.9]
# K = [4,5,6]

# from ProcessingData import ACOMP_CoAuthor
# dataset = ACOMP_CoAuthor
# # beta_center, beta_estimate, beta_actual = 0.6, 0.8, 0.8 #no good
# # beta_center, beta_estimate, beta_actual = 0.6, 0.8, 1 #no good
# beta_center, beta_estimate, beta_actual = 0.2, 0.3, 0.3

# name_method =["own_class_center","estimated_hyper_lin","actual_hyper_lin","distance_center_own_opposite_tam"]
# name_function = ["lin_center_own","exp","func_own_opp_new_v2"]

# time = datetime.now().strftime("%d%m%Y_%H%M%S")
# # filename = (str(dataset).split("\\")[-1]).split(".")[0]
# filename = 'ACOMP_19_G3_0.1_39_390_15'

# #W.svm
# for n in range(0,N):
#     header = ['Times','Fold','T','K','Name Method', 'Name Function', 'SP', 'SE', 'Gmean', 'F1 Score','Accuracy','AUC','Ma tran nham lan']
#     X, y = dataset.load_data()
#     print(X.shape)
#     kfold_validation = StratifiedKFold(n_splits=5, shuffle=True) 
#     with open(f'./Experiment/{filename}_{time}.csv', 'a', encoding='UTF8', newline='') as f3:
#         writer = csv.writer(f3)
#         writer.writerow(header)
#         fold = 1
#         for train_index, test_index in kfold_validation.split(X,y):
#             print("\nN = ",n+1,", Fold = ",fold)
#             # X_train, y_train = X.iloc[train_index], y.iloc[train_index]
#             # X_test, y_test = X.iloc[test_index], y.iloc[test_index]
#             X_train, y_train = X[train_index], y[train_index]
#             X_test, y_test = X[test_index], y[test_index]
#             # print(X_test.shape)
            
#             #Scalling Data
#             sc_X = StandardScaler()
#             X_train = sc_X.fit_transform(X_train)
#             X_test = sc_X.transform(X_test)
#             y_train = np.array(y_train)

#             # NORMAL

#             #Svm library
#             print("SVM LIBRARY starting...")
#             test_pred = svm_lib(X_train, y_train,X_test)
#             test_pred = np.nan_to_num(test_pred)
#             sp,se,gmean = Gmean(y_test,test_pred)
#             name1 = 'SVM'
#             name2 = 'SVM'
#             writer.writerow([n+1,fold,"None","None",name1,name2,sp,se,gmean,f1_score(y_test, test_pred),accuracy_score(y_test,test_pred),roc_auc_score(y_test, test_pred),str(confusion_matrix(y_test, test_pred))])
            
            
#             #Wsvm
#             print("W.SVM starting...")
#             N, d = X_train.shape
#             distribution_weight = np.ones(N)
#             test_pred = wsvm(C,X_train, y_train, X_test, distribution_weight)
#             test_pred = np.nan_to_num(test_pred)
#             sp,se,gmean = Gmean(y_test,test_pred)
#             name1 = 'WSVM'
#             name2 = 'WSVM'
#             writer.writerow([n+1,fold,"None","None",name1,name2,sp,se,gmean,f1_score(y_test, test_pred),accuracy_score(y_test,test_pred),roc_auc_score(y_test, test_pred),str(confusion_matrix(y_test, test_pred))])

#             #FuzyyWsvm
#             for namemethod in name_method:
#                 for namefunction in name_function:
#                     if namemethod =="distance_center_own_opposite_tam" and namefunction =="lin_center_own":
#                         continue
#                     elif namemethod =="distance_center_own_opposite_tam" and namefunction =="exp":
#                         continue
#                     elif namemethod == "own_class_center" and namefunction == "func_own_opp_new_v2":
#                         continue
#                     elif namemethod == "estimated_hyper_lin" and namefunction == "func_own_opp_new_v2":
#                         continue
#                     elif namemethod == "actual_hyper_lin" and namefunction == "func_own_opp_new_v2":
#                         continue
#                     else:
#                         print(f"Fuzzy W.SVM name_method = '{namemethod}',name_function = '{namefunction}' starting...")
#                         distribution_weight = fuzzy_weight(beta_center, beta_estimate, beta_actual,X_train, y_train,namemethod,namefunction)
#                         __ = fuzzy_weight(beta_center, beta_estimate, beta_actual,X_train, y_train,namemethod,namefunction)
#                         test_pred = wsvm(C,X_train, y_train, X_test, distribution_weight)
#                         test_pred = np.nan_to_num(test_pred)
#                         sp,se,gmean = Gmean(y_test,test_pred)
#                         writer.writerow([n+1,fold,"None","None",namemethod,namefunction,sp,se,gmean,f1_score(y_test, test_pred),accuracy_score(y_test,test_pred),roc_auc_score(y_test, test_pred),str(confusion_matrix(y_test, test_pred))])

#             ############ USING AFW-CIL METHOD #########################
#             print("===========USING AFW-CIL=====================")
#             arr_tlp = is_tomek_new(X_train, y_train, class_type = [-1.0])
#             #FuzyyWsvm
#             for namemethod in name_method:
#                 for namefunction in name_function:
#                     if namemethod =="distance_center_own_opposite_tam" and namefunction =="lin_center_own":
#                         continue
#                     elif namemethod =="distance_center_own_opposite_tam" and namefunction =="exp":
#                         continue
#                     elif namemethod == "own_class_center" and namefunction == "func_own_opp_new_v2":
#                         continue
#                     elif namemethod == "estimated_hyper_lin" and namefunction == "func_own_opp_new_v2":
#                         continue
#                     elif namemethod == "actual_hyper_lin" and namefunction == "func_own_opp_new_v2":
#                         continue
#                     else:
#                         print(f"Fuzzy W.SVM name_method = '{namemethod}',name_function = '{namefunction}' starting...")
#                         distribution_weight = fuzzy_weight(beta_center, beta_estimate, beta_actual,X_train, y_train,namemethod,namefunction)
#                         __ = fuzzy_weight(beta_center, beta_estimate, beta_actual,X_train, y_train,namemethod,namefunction)
                        
#                         for i in range(0,T):
#                             print("T = ",i+1)
#                             for n_neighbor in K:
#                                 new_W = data_tomelinks_new1(C,distribution_weight,X_test,y_test,X_train,y_train,n_neighbor,arr_tlp,clf=None,namemethod=namemethod,namefunction=namefunction)
#                                 test_pred = wsvm(C,X_train, y_train, X_test, new_W)
#                                 test_pred = np.nan_to_num(test_pred)
#                                 sp,se,gmean = Gmean(y_test,test_pred)
#                                 writer.writerow([n+1,fold,i+1,n_neighbor,namemethod,namefunction,sp,se,gmean,f1_score(y_test, test_pred),accuracy_score(y_test,test_pred),roc_auc_score(y_test, test_pred),str(confusion_matrix(y_test, test_pred))])
#             fold = fold + 1

In [15]:

def compare_arrays(array1, array2):
    different_positions = []
    for i in range(len(array1)):
        if array1[i] < array2[i]:
            different_positions.append(i)
    print(len(different_positions))
    return different_positions


In [16]:
# ###################### TEST-SIZE SCRIPT FOR CO-AUTHOR DATASETS #################################
# C = 100
# T = 20
# N = 1
# n_neighbor = [4,5,6]
# test_size = [0.2]
# new_rate = [1/5]
# from Processing_Data import CoAuthor_600, CoAuthor_1800
# dataset = CoAuthor_1800
# beta_center, beta_estimate, beta_actual = 0.3, 0.6, 0.7

# name_method =["own_class_center","estimated_hyper_lin","actual_hyper_lin","distance_center_own_opposite_tam"]
# name_function = ["lin_center_own","exp","func_own_opp_new", "func_own_opp_new_v1","func_own_opp_new_v2"]
# time = datetime.now().strftime("%d%m%Y_%H%M%S")
# filename = (str(dataset).split("\\")[-1]).split(".")[0]


# for n in range(0,N):
#     print("Lần bốc thứ: ",n+1)
#     header = ['Lan boc','Test Size','Rate','T','K-Neighbors','Name Method', 'Name Function', 'SP', 'SE', 'Gmean', 'F1 Score','Accuracy','AUC','Ma tran nham lan']
#     data = []
#     for testsize in test_size:
#         for newrate in new_rate:
#             X_train, y_train, X_test, y_test = dataset.load_data(test_size=testsize, new_rate=newrate)
#             with open(f'./Experiment/TestSize_Data_{filename}_{time}.csv', 'a', encoding='UTF8', newline='') as f3:
#                 writer = csv.writer(f3)
#                 writer.writerow(header)
                
#                 #Scalling Data
#                 sc_X = StandardScaler()
#                 X_train = sc_X.fit_transform(X_train)
#                 X_test = sc_X.transform(X_test)
#                 y_train = np.array(y_train)

#                 # NORMAL

#                 #Svm library
#                 print("SVM LIBRARY starting...\n")
#                 test_pred = svm_lib(X_train, y_train,X_test)
#                 sp,se,gmean = Gmean(y_test,test_pred)
#                 name1 = 'SVM'
#                 name2 = 'SVM'
#                 data.append([n+1,testsize,newrate,"None","None",name1,name2,sp,se,gmean,f1_score(y_test, test_pred),accuracy_score(y_test,test_pred),roc_auc_score(y_test, test_pred),str(confusion_matrix(y_test, test_pred))])
                
#                 #Wsvm
#                 print("W.SVM starting...\n")
#                 N, d = X_train.shape
#                 distribution_weight = np.ones(N)
#                 test_pred = wsvm(C,X_train, y_train, X_test, distribution_weight)
#                 sp,se,gmean = Gmean(y_test,test_pred)
#                 name1 = 'WSVM'
#                 name2 = 'WSVM'
#                 data.append([n+1,testsize,newrate,"None","None",name1,name2,sp,se,gmean,f1_score(y_test, test_pred),accuracy_score(y_test,test_pred),roc_auc_score(y_test, test_pred),str(confusion_matrix(y_test, test_pred))])

#                 #FuzyyWsvm
#                 for namemethod in name_method:
#                     for namefunction in name_function:
#                         if namemethod =="distance_center_own_opposite_tam" and namefunction =="lin_center_own":
#                             continue
#                         elif namemethod =="distance_center_own_opposite_tam" and namefunction =="exp":
#                             continue
#                         elif namemethod == "own_class_center" and namefunction == "func_own_opp_new":
#                             continue
#                         elif namemethod == "estimated_hyper_lin" and namefunction == "func_own_opp_new":
#                             continue
#                         elif namemethod == "actual_hyper_lin" and namefunction == "func_own_opp_new":
#                             continue
#                         elif namemethod == "own_class_center" and namefunction == "func_own_opp_new_v1":
#                             continue
#                         elif namemethod == "estimated_hyper_lin" and namefunction == "func_own_opp_new_v1":
#                             continue
#                         elif namemethod == "actual_hyper_lin" and namefunction == "func_own_opp_new_v1":
#                             continue
#                         elif namemethod == "own_class_center" and namefunction == "func_own_opp_new_v2":
#                             continue
#                         elif namemethod == "estimated_hyper_lin" and namefunction == "func_own_opp_new_v2":
#                             continue
#                         elif namemethod == "actual_hyper_lin" and namefunction == "func_own_opp_new_v2":
#                             continue
#                         else:
#                             print(f"Fuzzy W.SVM name_method = '{namemethod}',name_function = '{namefunction}' starting...\n")
#                             distribution_weight = fuzzy_weight(beta_center, beta_estimate, beta_actual,X_train, y_train,namemethod,namefunction)
#                             test_pred = wsvm(C,X_train, y_train, X_test, distribution_weight)
#                             sp,se,gmean = Gmean(y_test,test_pred)
#                             data.append([n+1,testsize,newrate,"None","None",namemethod,namefunction,sp,se,gmean,f1_score(y_test, test_pred),accuracy_score(y_test,test_pred),roc_auc_score(y_test, test_pred),str(confusion_matrix(y_test, test_pred))])

#                 ############ USING AFW-CIL METHOD #########################
#                 arr_tlp = is_tomek_new(X_train, y_train, class_type = [-1.0])
#                 print(arr_tlp)
#                 #FuzyyWsvm
#                 for namemethod in name_method:
#                     for namefunction in name_function:
#                         if namemethod =="distance_center_own_opposite_tam" and namefunction =="lin_center_own":
#                             continue
#                         elif namemethod =="distance_center_own_opposite_tam" and namefunction =="exp":
#                             continue
#                         elif namemethod == "own_class_center" and namefunction == "func_own_opp_new":
#                             continue
#                         elif namemethod == "estimated_hyper_lin" and namefunction == "func_own_opp_new":
#                             continue
#                         elif namemethod == "actual_hyper_lin" and namefunction == "func_own_opp_new":
#                             continue
#                         elif namemethod == "own_class_center" and namefunction == "func_own_opp_new_v1":
#                             continue
#                         elif namemethod == "estimated_hyper_lin" and namefunction == "func_own_opp_new_v1":
#                             continue
#                         elif namemethod == "actual_hyper_lin" and namefunction == "func_own_opp_new_v1":
#                             continue
#                         elif namemethod == "own_class_center" and namefunction == "func_own_opp_new_v2":
#                             continue
#                         elif namemethod == "estimated_hyper_lin" and namefunction == "func_own_opp_new_v2":
#                             continue
#                         elif namemethod == "actual_hyper_lin" and namefunction == "func_own_opp_new_v2":
#                             continue
#                         else:
#                             print("===========USING AFW-CIL=====================")
#                             print(f"Fuzzy W.SVM name_method = '{namemethod}',name_function = '{namefunction}' starting...\n")
#                             distribution_weight = fuzzy_weight(beta_center, beta_estimate, beta_actual,X_train, y_train,namemethod,namefunction)                  
                            
#                             for i in range(0,T):
#                                 print("T = ",i+1)
#                                 for nb in n_neighbor:
#                                     print("K-neighbors = ",nb)
#                                     new_W = data_tomelinks_new1(C,distribution_weight,X_test,y_test,X_train,y_train,nb,arr_tlp,clf=None,namemethod=namemethod,namefunction=namefunction)
#                                     # print(arr_tlp)
#                                     # print(new_W)
#                                     test_pred = wsvm(C,X_train, y_train, X_test, new_W)
#                                     sp,se,gmean = Gmean(y_test,test_pred)
#                                     data.append([n+1,testsize,newrate,i+1,nb,namemethod,namefunction,sp,se,gmean,f1_score(y_test, test_pred),accuracy_score(y_test,test_pred),roc_auc_score(y_test, test_pred),str(confusion_matrix(y_test, test_pred))])
#                 writer.writerows(data)


In [17]:
# ###################### TEST-SIZE SCRIPT FOR YEAST DATASETS #################################
# C = 100
# T = 3
# N = 1
# n_neighbor = [5]
# test_size = [0.3]

# from Processing_Data import Yeast, Abanole_KFold
# dataset = Abanole_KFold
# beta_center, beta_estimate, beta_actual = 0.9, 0.2, 0.3

# name_method =["own_class_center","estimated_hyper_lin","actual_hyper_lin","distance_center_own_opposite_tam"]
# name_function = ["lin_center_own","exp","func_own_opp_new", "func_own_opp_new_v1","func_own_opp_new_v2"]
# time = datetime.now().strftime("%d%m%Y_%H%M%S")
# filename = (str(dataset).split("\\")[-1]).split(".")[0]


# for n in range(0,N):
#     print("Lần bốc thứ: ",n+1)
#     header = ['Lan boc','Test Size','T','K-Neighbors','Name Method', 'Name Function', 'SP', 'SE', 'Gmean', 'F1 Score','Accuracy','AUC','Ma tran nham lan']
#     data = []
#     for testsize in test_size:
#         X_train, y_train, X_test, y_test = dataset.load_data()
#         with open(f'./Experiment/TestSize_Data_{filename}_{time}.csv', 'a', encoding='UTF8', newline='') as f3:
#             writer = csv.writer(f3)
#             writer.writerow(header)
            
#             #Scalling Data
#             sc_X = StandardScaler()
#             X_train = sc_X.fit_transform(X_train)
#             X_test = sc_X.transform(X_test)
#             y_train = np.array(y_train)

#             # NORMAL

#             #Svm library
#             print("SVM LIBRARY starting...\n")
#             test_pred = svm_lib(X_train, y_train,X_test)
#             sp,se,gmean = Gmean(y_test,test_pred)
#             name1 = 'SVM'
#             name2 = 'SVM'
#             data.append([n+1,testsize,"None","None",name1,name2,sp,se,gmean,f1_score(y_test, test_pred),accuracy_score(y_test,test_pred),roc_auc_score(y_test, test_pred),str(confusion_matrix(y_test, test_pred))])
            
#             #Wsvm
#             print("W.SVM starting...\n")
#             N, d = X_train.shape
#             distribution_weight = np.ones(N)
#             test_pred = wsvm(C,X_train, y_train, X_test, distribution_weight)
#             sp,se,gmean = Gmean(y_test,test_pred)
#             name1 = 'WSVM'
#             name2 = 'WSVM'
#             data.append([n+1,testsize,"None","None",name1,name2,sp,se,gmean,f1_score(y_test, test_pred),accuracy_score(y_test,test_pred),roc_auc_score(y_test, test_pred),str(confusion_matrix(y_test, test_pred))])

#             #FuzyyWsvm
#             for namemethod in name_method:
#                 for namefunction in name_function:
#                     if namemethod =="distance_center_own_opposite_tam" and namefunction =="lin_center_own":
#                         continue
#                     elif namemethod =="distance_center_own_opposite_tam" and namefunction =="exp":
#                         continue
#                     elif namemethod == "own_class_center" and namefunction == "func_own_opp_new":
#                         continue
#                     elif namemethod == "estimated_hyper_lin" and namefunction == "func_own_opp_new":
#                         continue
#                     elif namemethod == "actual_hyper_lin" and namefunction == "func_own_opp_new":
#                         continue
#                     elif namemethod == "own_class_center" and namefunction == "func_own_opp_new_v1":
#                         continue
#                     elif namemethod == "estimated_hyper_lin" and namefunction == "func_own_opp_new_v1":
#                         continue
#                     elif namemethod == "actual_hyper_lin" and namefunction == "func_own_opp_new_v1":
#                         continue
#                     elif namemethod == "own_class_center" and namefunction == "func_own_opp_new_v2":
#                         continue
#                     elif namemethod == "estimated_hyper_lin" and namefunction == "func_own_opp_new_v2":
#                         continue
#                     elif namemethod == "actual_hyper_lin" and namefunction == "func_own_opp_new_v2":
#                         continue
#                     else:
#                         print(f"Fuzzy W.SVM name_method = '{namemethod}',name_function = '{namefunction}' starting...\n")
#                         distribution_weight = fuzzy_weight(beta_center, beta_estimate, beta_actual,X_train, y_train,namemethod,namefunction)
#                         test_pred = wsvm(C,X_train, y_train, X_test, distribution_weight)
#                         sp,se,gmean = Gmean(y_test,test_pred)
#                         data.append([n+1,testsize,"None","None",namemethod,namefunction,sp,se,gmean,f1_score(y_test, test_pred),accuracy_score(y_test,test_pred),roc_auc_score(y_test, test_pred),str(confusion_matrix(y_test, test_pred))])

#             ############ USING AFW-CIL METHOD #########################
#             arr_tlp = is_tomek_new(X_train, y_train, class_type = [-1.0])
#             print(arr_tlp)
#             #FuzyyWsvm
#             for namemethod in name_method:
#                 for namefunction in name_function:
#                     if namemethod =="distance_center_own_opposite_tam" and namefunction =="lin_center_own":
#                         continue
#                     elif namemethod =="distance_center_own_opposite_tam" and namefunction =="exp":
#                         continue
#                     elif namemethod == "own_class_center" and namefunction == "func_own_opp_new":
#                         continue
#                     elif namemethod == "estimated_hyper_lin" and namefunction == "func_own_opp_new":
#                         continue
#                     elif namemethod == "actual_hyper_lin" and namefunction == "func_own_opp_new":
#                         continue
#                     elif namemethod == "own_class_center" and namefunction == "func_own_opp_new_v1":
#                         continue
#                     elif namemethod == "estimated_hyper_lin" and namefunction == "func_own_opp_new_v1":
#                         continue
#                     elif namemethod == "actual_hyper_lin" and namefunction == "func_own_opp_new_v1":
#                         continue
#                     elif namemethod == "own_class_center" and namefunction == "func_own_opp_new_v2":
#                         continue
#                     elif namemethod == "estimated_hyper_lin" and namefunction == "func_own_opp_new_v2":
#                         continue
#                     elif namemethod == "actual_hyper_lin" and namefunction == "func_own_opp_new_v2":
#                         continue
#                     else:
#                         print("===========USING AFW-CIL=====================")
#                         print(f"Fuzzy W.SVM name_method = '{namemethod}',name_function = '{namefunction}' starting...\n")
#                         distribution_weight = fuzzy_weight(beta_center, beta_estimate, beta_actual,X_train, y_train,namemethod,namefunction)                  
                        
#                         for i in range(0,T):
#                             print("T = ",i+1)
#                             for nb in n_neighbor:
#                                 print("K-neighbors = ",nb)
#                                 new_W = data_tomelinks_new1(C,distribution_weight,X_test,y_test,X_train,y_train,nb,arr_tlp,clf=None,namemethod=namemethod,namefunction=namefunction)
#                                 # print(arr_tlp)
#                                 # print(new_W)
#                                 test_pred = wsvm(C,X_train, y_train, X_test, new_W)
#                                 sp,se,gmean = Gmean(y_test,test_pred)
#                                 data.append([n+1,testsize,i+1,nb,namemethod,namefunction,sp,se,gmean,f1_score(y_test, test_pred),accuracy_score(y_test,test_pred),roc_auc_score(y_test, test_pred),str(confusion_matrix(y_test, test_pred))])
#             writer.writerows(data)


In [18]:
f3.close()